In [36]:
import json

# Q1

In [45]:
%%capture elastic_config
!curl localhost:9200

In [46]:
elastic_config = json.loads(elastic_config.stdout)
elastic_config['version']['build_hash']

'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7'

# Q2

In [48]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [51]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [50]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [52]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [53]:
from tqdm.auto import tqdm

/Users/vpankrat/Courses/LLMzoomcamp/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:06<00:00, 144.09it/s]


# Q3

In [74]:
query = "How do execute a command on a Kubernetes pod?"

In [75]:
def elastic_max_score(query):
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": { # text matching
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"], # question is 3 times more important than other fields
                        "type": "best_fields"
                    }
                },
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    print(response['hits']['max_score'])

In [76]:
elastic_max_score(query)

44.50556


# Q4

In [70]:
query = "How do copy a file to a Docker container?"

In [77]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": { # text matching
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # question is 3 times more important than other fields
                        "type": "best_fields"
                    }
                },
                "filter": { # filtering component
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [80]:
[el['question'] for el in elastic_search(query)][2]

'How do I copy files from a different folder into docker container’s working directory?'

# Q5

In [96]:
question = "How do I execute a command in a running docker container?"

In [97]:
q4_res = elastic_search(query)
q4_res

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [86]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [90]:
context_parts = []
for el in q4_res:
    context_parts.append(context_template.format(**el))
context = '\n\n'.join(context_parts)
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: Kubernetes-dashboard
A: Deploy and Access the Kubernetes Dashboard
Luke

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:
COPY ["src/predict.py", "models/xgb_model.bin", "./"]											Gopakumar Gopinathan


In [92]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [94]:
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: Kubernetes-dashboard
A: Deploy and Access the Kubernetes Dashboard
Luke

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic synta

In [95]:
len(prompt)

1101

# Q6

In [98]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tiktoken]


In [100]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [104]:
tokens = encoding.encode(prompt)
tokens

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 8470,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 127933,
 120341,
 198,
 32,
 25,
 140211,
 326,
 12993,
 290,
 127933,
 45324,
 198,
 100418,

In [105]:
encoding.decode_single_token_bytes(50738)

b'TEXT'

In [106]:
len(tokens)

242

# Bonus: generating the answer (ungraded)

In [107]:
import os
import yaml
from openai import AzureOpenAI

In [108]:
from dataclasses import dataclass

@dataclass
class LLMServiceConfig:
    host: str
    api_key: str
    api_version: str
    model: str

with open("../OPENAI_API_KEY.yaml") as f:
    details = yaml.safe_load(f)["Glossary Terms Extraction Service"]
    
service_config = LLMServiceConfig(
    host=f"{details['protocol']}://{details['host']}",
    api_key=details['api_key'],
    api_version=details['api_version'],
    model=details['model']
)
client = AzureOpenAI(
    api_version=service_config.api_version,
    azure_endpoint=service_config.host,
    api_key=service_config.api_key
)

In [109]:
response = client.chat.completions.create(
    model=service_config.model,
    messages=[
        {'role': 'user', 'content': prompt}
    ]
)

In [111]:
print(response.choices[0].message.content)

To execute a command in a running Docker container, follow these steps:

1. Identify the container ID of the running container using the command:
   ```
   docker ps
   ```

2. Use the `docker exec` command to execute a specific command in the container. For example, to start a bash session inside the container:
   ```
   docker exec -it <container-id> bash
   ```


# Bonus: calculating the costs (ungraded)

In [112]:
one_req_price = (150/1000 * 0.005 + 250/1000 * 0.015) 
one_req_price

0.0045

In [113]:
thousand_req_price = one_req_price * 1000
thousand_req_price

4.5